In [1]:
library(EDASeq)
library(RUVSeq)
library(SCnorm)
library(sva)
library(scRNA.seq.funcs)

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which.min

Welcome to Bioconductor

    

In [2]:
combat <- function(
    mat,
    batchinfo_path,
    output_path
){
    print('start batch removal using combat')
    batch_info <-read.table(batchinfo_path,sep='\t',row.names = 1, header=T,check.names = FALSE)
    foo <<- batch_info
    batch_info[,1] <- factor(batch_info[,1])
    if (!(dim(mat)[2]==dim(batch_info)[1]))
        stop('sample numbers in batch info and expression matrix should be same')
    batchname <-toString(names(batch_info)[1])
    batch_info=batch_info[names(mat),]
    mod <- model.matrix(~ 1, data = batch_info)
    combat <- ComBat(
        dat = log(mat+0.001),
        batch = batch_info[,1],
        mod = mod,
        par.prior = TRUE,
        prior.plots = FALSE
    )
    
    mat <- exp(combat)
    write.csv(mat, file=paste(output_path,'mx_combat_batchremoval_',batchname,'.txt',sep=''))
    #mat
}

In [81]:
combat(TMM_data, "/home/xieyufeng/fig3/data/cfRNA/batch_info.txt", "~/")

[1] "start batch removal using combat"


ERROR: Error in `[.default`(batch_info, , 1): incorrect number of dimensions


In [84]:
foo

,dataset
17402567-B,lulab_hcc
249136-B,lulab_hcc
385247-B,lulab_hcc
497411-B,lulab_hcc
498221-B,lulab_hcc
507450-B,lulab_hcc
507511-B,lulab_hcc
507867-B,lulab_hcc
507887-B,lulab_hcc
508877-B,lulab_hcc


In [3]:
ruv <- function(
    mat,
    classinfo_path,
    output_path,
    label_column = 2,
    k = 10
){
    print('start batch removal using RUVs')
    cIdx <- rownames(mat)
    
    sample_info <- read.table(classinfo_path,sep='\t',header=TRUE,  check.names=FALSE,  stringsAsFactors=FALSE)
    ##rank by mat
    if(unique(is.na(sample_info$sample_id))) 
    stop("sample_id not in file")
    rownames(sample_info) = sample_info$sample_id
    sample_info=sample_info[names(mat),]
    rownames(sample_info) <- c()
    
    names(sample_info)[label_column]="label"
    scIdx <- matrix(-1, ncol = max(table(sample_info$label)), nrow = dim(table(sample_info$label)))
    labellist <- names(table(sample_info$label))
    for(i in c(1:dim(table(sample_info$label)))) {
        tmp <- which(sample_info$label == labellist[i])
        scIdx[i, 1:length(tmp)] <- tmp
    }
    mat <- log(mat+0.001)
    ruv <- RUVs(as.matrix(mat), cIdx, k = k, scIdx = scIdx, isLog = TRUE)
    write.table(exp(ruv$normalizedCounts), file=paste(output_path, 'mx_ruv_batchremoval.txt',sep=''), sep='\t', quote=FALSE, row.names=TRUE, col.names=TRUE)
}

In [4]:
TMM_data <- read.table("/home/xieyufeng/fig3/output/merge/matrix_processing/filter.null.Norm_TMM.mirna_and_domains.txt", 
                       header = T, check.names=FALSE,  stringsAsFactors=FALSE, row.names = 1)
ruv(TMM_data, "/home/xieyufeng/fig3/data/merge/sample_classes.txt", "~/fig3/batch_correction/")

[1] "start batch removal using RUVs"


In [65]:
combat(TMM_data, "/home/xieyufeng/fig3/data/cfRNA/batch_info.txt", "~/")

[1] "start batch removal using combat"


ERROR: Error in `[.default`(batch_info, , 1): incorrect number of dimensions
